In [1]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm
import numdifftools as nd #Using this package to numerically approximate the hessian matrix

In [3]:
#Locate my folder
my_folder='/home/august/code/jupyter/Econometrics/'
#C:\Users\vjule\OneDrive\Dokumenter\skole\Kandidatfag\Financial_econometrics\Undervisning\Assignments
y_val = genfromtxt(my_folder+'xbtusd.csv', delimiter=',') #loading the csv data and storing it as a np array
Y = y_val[:, 3:4] #Taking the relevant retrended column from the CSV-file 
Y=Y[1:] #First observation is a NaN value and we get rid of that 
T=len(Y) #Finding length of the column which we use to reshape (right now it is a (TX1) np array, I want a 1XT even though the code might still work for the former)
Y=np.reshape(Y,T) #Getting a 1XT np array (so a "standard" np.array with our observations)

In [4]:
def DAR_likelihood(phi, omega, alpha,):
    T=len(Y)
    
    #Define the series sigma^2_t (sigma2), e_t=x_t-phi*x_{t-1} (e) as empty variables we assign values to later. 
    sigma2=np.zeros(T)
    e=np.zeros(T)
    logli=np.zeros(T)
    
    for t in range(1,T):
        sigma2[t]=omega + alpha*Y[t-1]**2
        e[t]=Y[t]-phi*Y[t-1]
        logli[t]=-0.5*np.log(2*np.pi)-0.5*np.log(sigma2[t])-0.5*e[t]**2/sigma2[t]
    
    LogL=-np.sum(logli)
    #Note we take the expression and multiply with (-1) as we are going to minimize the function in the optimization
    
    return LogL

def optim(x): #defining an optimization function that only can change phi,omega and alpha
    ph,ome,alp=x 
    return DAR_likelihood(phi=ph, omega=ome,alpha=alp)

In [5]:
#Maximization - 
#Note on the optimization: I wish to highlight that the optimazation of the DAR-likelihood is rather complex and some
# - optimizers (like Newton-CG and L-BFGS-B) have an issue getting satisfying convergence. We use SLSQP which is ideal
#- for mathematical problems for which the objective function and the constraints are twice continuously differentiable.
#- Nevertheless, when using SLSQP we can't get the inverse hessian directly from the computation (from scipy) which we used
#- for the ARCH(1) optimizations. In order to get the hessian and thereby the standard errors of the estimates we use
#- numdifftools which approximate the hessian numerically. 
Par0=np.array([0.0,0.05,0.5]) #initial guesses
res=opt.minimize(optim, Par0, method='SLSQP', bounds=((0.0001,None),(0,None),(None,None))) #optimizing using the SLSQP (sequential least squares programming) algorithm

/tmp/ipykernel_38291/3715273022.py:12: RuntimeWarning: invalid value encountered in log
  logli[t]=-0.5*np.log(2*np.pi)-0.5*np.log(sigma2[t])-0.5*e[t]**2/sigma2[t]


In [6]:
res.x

array([ 0.9014213 , 47.66459993,  0.18591273])

In [7]:
H = nd.Hessian(optim)([0.9014295 , 47.66480794,  0.18590847]) #Approximating the hessian numerically evaluated at the parameter estimates
#Neglect the warning message in this case

In [8]:
V_H=np.linalg.inv(H) #Getting the inverse hessian
se_H=np.sqrt(np.diagonal(V_H)) #Getting the standard errors from the inverse hessian

In [9]:
phi_est=res.x[0]
omega_est=res.x[1]
alpha_est=res.x[2]
print('Estimates are given by with std. errors and t_vals in ():')
print('phi_bar_est= '+str(phi_est)+', ( se(.)=' +str(se_H[0])+', t_val= ' +str(phi_est/se_H[0])+' )' )
print('omega_est= '+str(omega_est)+', ( se(.)=' +str(se_H[1])+', t_val= ' +str(omega_est/se_H[1])+' )' )
print('alpha_est= '+str(alpha_est)+', ( se(.)=' +str(se_H[2])+', t_val= ' +str(alpha_est/se_H[2])+' )' )

Estimates are given by with std. errors and t_vals in ():
phi_bar_est= 0.901421296462886, ( se(.)=0.07182136236120729, t_val= 12.550879944735895 )
omega_est= 47.664599932661915, ( se(.)=8.618364650412293, t_val= 5.530585194069468 )
alpha_est= 0.18591273412577303, ( se(.)=0.04983055783670401, t_val= 3.7308981114563022 )
